In [1]:
import time
startTime=time.time()
import os
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#参数
image_shape=(256,256,3)
kernel_size=3 #数值7效果差 ，数值3、4、5效果好
dropout_ratio=0.2
batch_size=54 #查看显卡信息：nvidia-smi
epochs=1000 

In [3]:
import tensorflow
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()
model.add(layers.Conv2D(4, kernel_size=kernel_size, padding="same", activation='relu',input_shape=image_shape)) 
#model.add(layers.BatchNormalization()) #效果不好，函数式API中放到激活函数之前也是效果不好
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(dropout_ratio))
model.add(layers.Conv2D(4, kernel_size=kernel_size, padding="same", activation='relu'))
#model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(dropout_ratio))
model.add(layers.Conv2D(8, kernel_size=kernel_size, padding="same", activation='relu'))
#model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(dropout_ratio))
model.add(layers.Conv2D(8, kernel_size=kernel_size, padding="same", activation='relu'))
#model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(dropout_ratio))
model.add(layers.Conv2D(16, kernel_size=kernel_size, padding="same", activation='relu'))
#model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(dropout_ratio))
model.add(layers.Conv2D(16, kernel_size=kernel_size, padding="same", activation='relu'))
#model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(dropout_ratio)) #仅一层Dropout效果不好，需要每层Dropout

model.add(layers.Flatten())
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 4)       112       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 4)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 128, 128, 4)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 4)       148       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 4)        0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 64, 64, 4)         0

In [4]:
from tensorflow.keras import optimizers
model.compile(loss='binary_crossentropy',
            optimizer=optimizers.Adam(),
            metrics=['acc'])

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
train_dir='.\\dataset\\train'
train_generator = train_datagen.flow_from_directory(
                                                    train_dir,
                                                    target_size=(image_shape[0], image_shape[1]),
                                                    batch_size=batch_size,
                                                    class_mode='binary')
print('train_generator.class_indices=',train_generator.class_indices) #train_generator.class_indices= {'stable': 0, 'unstable': 1}



validation_datagen = ImageDataGenerator(rescale=1./255)
validation_dir='.\\dataset\\validation'
validation_generator = validation_datagen.flow_from_directory(
                                                    validation_dir,
                                                    target_size=(image_shape[0], image_shape[1]),
                                                    batch_size=batch_size,
                                                    class_mode='binary')
print('validation_generator.class_indices=',validation_generator.class_indices) 


Found 8748 images belonging to 2 classes.
train_generator.class_indices= {'stable': 0, 'unstable': 1}
Found 4374 images belonging to 2 classes.
validation_generator.class_indices= {'stable': 0, 'unstable': 1}


In [7]:
callbacks_list = [tensorflow.keras.callbacks.ModelCheckpoint(filepath="./checkpoint/checkpoint{epoch:03d}.ckpt")]
history = model.fit(
                                train_generator,
                                epochs=epochs,
                                callbacks=callbacks_list,
                                validation_data=validation_generator, #验证耗时有限
                                )

model.save('01.h5')
endTime=time.time()
print('How many minutes:',(endTime-startTime)/60)

Epoch 1/1000


KeyboardInterrupt: 

In [ ]:
validation_loss, validation_acc = model.evaluate(validation_generator)
print('validation_acc=',validation_acc)

train_loss, train_acc = model.evaluate(train_generator)
print('train_acc=',train_acc)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_dir='.\\dataset\\test'
test_generator = test_datagen.flow_from_directory(
                                                    test_dir,
                                                    target_size=(image_shape[0], image_shape[1]),
                                                    batch_size=batch_size,
                                                    class_mode='binary')
print('test_generator.class_indices=',test_generator.class_indices) 

test_loss, test_acc = model.evaluate(test_generator)
print('test_acc=',test_acc)

In [ ]:
with open(os.path.join('history.txt'), 'w') as file_object:
    file_object.write(str(history.history))

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()